Data Modeling
===

Prediction the targets based on patient and biomarker data.

### Model

What model should we use for this modeling problem?
We have an ambiguous classification problem with unclear intended use-cases.
Further, the data contains a large number of possible raw features, contains some missing data, and is structured.
This is an excellent fit for gradient boosting machines (GBMs).
The most prominent GBM implementations are LightGBM and XGBoost, but we'll just use the newer scikit-learn implementation (which is based on and benchmarks competitively with LightGBM).

#### Other options
Future work could explore other model classes, particularly if a particular use-case was identified for which a different model would be a better fit.

A few options worth considering:

 - Models like logistic regression have the benefit of familiarity to clinical audiences. Alternately, presenting the correlations or odds ratios for just a few of the patient variables could be useful for some audiences (at the cost of predictive power).
 - Alternative models could include other highly intepretable models. For example, an integer-only risk-scoring model (e.g. fit using [FasterRisk](https://github.com/jiachangliu/FasterRisk)) could be worth exploring.
 - Deep learning models could be used to learn separate embeddings for patients and biomarkers. These embeddings could be useful for more than just the given classification problem.

### Data and modeling features

Most of the feature visualization and development work is in the `DataExploration` notebook (in this same directory).

#### Other options

### Evaluation and metrics

A downside to this approach is that folds have non-consistent sizes, making comparison between models challenging.
In other words, the training set will be larger when predicting on patients at the smallest institutions.

#### Other options

### Model optimization and hyperparameter tuning

I trained all models on a single CPU core of a Macbook Air.
Therefore, I both didn't parellelize hyperparameter tuning (due to low available memory) and didn't train very many models during hyperparameter tuning.
As a result, I was selective about the experiments that I ran and the parameters I varied together.
Following [Google's tuning guidance](https://github.com/google-research/tuning_playbook#a-scientific-approach-to-improving-model-performance), I used iterative tuning within hand-designed parameter search spaces. 
That's an unneccessarily fancy way of 

Here are some things I would have liked to do but didn't get around to doing:
 - My validation approach (20-fold cross validation) is overly simplistic; realistically there is leakage between train and test, as patient hospitalizations are not independent events. A minimally reasonable approach, and the reason the `hdf` below contains provider information, would be to create 1 test fold for each provider. Creating a single temporally-bounded test set could be reasonable as well, although it would have to be done carefully.
 - Given the small number of covariates, 
 - The hyperparameter search I implemented is very cursory. Given the small size of the dataset, doing a more detailed search is probably overkill, but I would still expect to use a more elaborate test harness in situations where prediction is important.
 - The definition of hospitalizations I used (as implemented in `covid_modeling.io.DataLoader`) is probably questionable; modeling in a situation like this (where I have no knowledge of the data-generating process) is fairly unreasonable.  If there were no mechanism to better understand the data-generating process (such as via a SME or dataset documentation), drawing conclusions from this data would be unethical (and unwarranted).  In particular, noise in the application of SNOMED CT codes is something I mention in the `DataExploration` notebook, but I ultimately decided to be out-of-scope for this particular exploration. (For example, one might try to impute missing reason-codes or correct erroneous reason codes where there is reason to suspect a COVID-19 diagnosis.)
 - "Interpretable" is always contextual. I chose logistic regression because it is a language familiar to many data scientists and subject-matter experts---potentially even more familiar than a decision tree!  However, for particular types of clinical practitioners, I would try to create a much simpler model (e.g. using only a single covariate, or the integer-only model mentioned above).


In [2]:
import json
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

import bcs.data_loader
import bcs.modeling

In [3]:
data_dir = (Path.cwd() / ".." / "data").resolve()
assert data_dir.exists()
figures_dir = (Path.cwd() / ".." / "figures").resolve()
figures_dir.mkdir(exist_ok=True)

In [4]:
dl = bcs.data_loader.DataLoader(data_dir)

In [5]:
pdf = dl.get_patient_dataframe()
bdf = dl.get_biomarker_dataframe()
tdf = dl.get_target_dataframe()

In [6]:
mdf = pd.merge(tdf, pdf, how="left", left_on="patient_id", right_index=True)
mdf = pd.merge(mdf, bdf, how="left", left_on="biomarker_id", right_index=True)
mdf.shape

(1734, 15174)

In [6]:
# config = bcs.modeling.ModelingConfig()
# model_evaluator = bcs.modeling.ModelEvaluator(config, mdf)

In [18]:
model_map = {}

In [19]:
bcs.modeling.ModelingConfig(**{"experiment_name": "test"})

ModelingConfig(experiment_name='test', patient_feature_action='keep', biomarker_feature_action='keep', biomarker_svd_n_components=None, is_disease_sub_type_ordered=True, excluded_columns=['status_alcohol_usage', 'status_exercise_frequency', 'status_bmi_level', 'status_days_since_diagnosis'], cv_column='institution_name', target_column='target_label', gbm_learning_rate=0.1, gbm_max_iter=100, gbm_max_leaf_nodes=31, gbm_min_samples_leaf=20, gbm_l2_regularization=0.0)

In [ ]:
configs = []
named_models = [
    {
        "experiment_name": "all",
    },
    {
        "experiment_name": "patient_only",
        "biomarker_feature_action": "exclude",
    },
    {
        "experiment_name": "patient_only_unorderedDisease",
        "biomarker_feature_action": "exclude",
        "is_disease_sub_type_ordered": False,
    },
    {
        "experiment_name": "biomarker_only",
        "patient_feature_action": "exclude",
    },
    {
        "experiment_name": "biomarker_only_svd",
        "patient_feature_action": "exclude",
        "biomarker_feature_action": "svd",
    },
]
for named_model in named_models
    config = bcs.modeling.ModelingConfig(
        gbm_learning_rate=0.2,
        gbm_max_iter=200,
        gbm_max_leaf_nodes=31,
        gbm_min_samples_leaf=10,
        gbm_l2_regularization=0.01,
        **named_model,
    )
    configs.append(config)
assert len(configs) == len(set([c.experiment_name for c in configs])), "Experiment names not unique."
len(configs)

In [ ]:
#
for config in tqdm(configs):
    if config.experiment_name in model_map:
        continue
    else:
        model_evaluator = bcs.modeling.ModelEvaluator(config, mdf)
        model = model_evaluator.train_and_evaluate()
        model_map[config.experiment_name] = model

len(model_map)

## Hyperparameter tuning

These experiments take a while to run (as the tqdm output below suggests), so they are maintained here as a record but not intended to be executed in normal cell execution flow. They were used to set the values used in the configurations above.

In [6]:
configs = []
for gbm_learning_rate in [0.1, 0.2]:
    for gbm_max_iter in [100, 200]:
        for gbm_max_leaf_nodes in [31, 63]:
            for gbm_min_samples_leaf in [10, 20]:
                for gbm_l2_regularization in [0.0, 0.02]:
                    config = bcs.modeling.ModelingConfig(
                        experiment_name=f"gbm_lr{gbm_learning_rate:.1f}_it{gbm_max_iter}_max{gbm_max_leaf_nodes}_min{gbm_min_samples_leaf}_r{gbm_l2_regularization:.2f}",
                        gbm_learning_rate=gbm_learning_rate,
                        gbm_max_iter=gbm_max_iter,
                        gbm_max_leaf_nodes=gbm_max_leaf_nodes,
                        gbm_min_samples_leaf=gbm_min_samples_leaf,
                        gbm_l2_regularization=gbm_l2_regularization,
                        biomarker_feature_action="exclude",
                    )
                    configs.append(config)
assert len(configs) == len({c.experiment_name for c in configs}), "Experiment names not unique."
len(configs)

32

In [12]:
configs = []
for gbm_learning_rate in [0.1, 0.2]:
    for gbm_max_iter in [100, 200]:
        for gbm_max_leaf_nodes in [31, 63]:
            for gbm_min_samples_leaf in [10, 20]:
                config = bcs.modeling.ModelingConfig(
                    experiment_name=f"gbm_lr{gbm_learning_rate:.1f}_it{gbm_max_iter}_max{gbm_max_leaf_nodes}_min{gbm_min_samples_leaf}",
                    gbm_learning_rate=gbm_learning_rate,
                    gbm_max_iter=gbm_max_iter,
                    gbm_max_leaf_nodes=gbm_max_leaf_nodes,
                    gbm_min_samples_leaf=gbm_min_samples_leaf,
                    gbm_l2_regularization=0.01,
                    biomarker_feature_action="keep",
                )
                configs.append(config)
assert len(configs) == len({c.experiment_name for c in configs}), "Experiment names not unique."
len(configs)

16

In [16]:
metrics_list = []
for config in tqdm(configs):
    model_evaluator = bcs.modeling.ModelEvaluator(config, mdf)
    model_metrics_list = model_evaluator.train_and_evaluate()
    metrics_list.extend(model_metrics_list)
len(metrics_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [3:05:01<00:00, 693.84s/it]


112

In [11]:
# hyperparameter tuning when biomarker features are excluded
# no longer captured in cell order
metrics = pd.DataFrame(metrics_list)
metrics.loc[metrics.institution_name == "All", ["experiment_name", "acc", "f1_pos", "roc_auc"]].sort_values(
    by="roc_auc", ascending=False
)

,experiment_name,acc,f1_pos,roc_auc
27,gbm_lr0.1_it100_max31_min20_r0.02,0.751442,0.273187,0.606639
20,gbm_lr0.1_it100_max31_min20_r0.00,0.737601,0.242928,0.605682
48,gbm_lr0.1_it100_max63_min20_r0.00,0.742215,0.258706,0.603535
13,gbm_lr0.1_it100_max31_min10_r0.02,0.751442,0.282862,0.601420
6,gbm_lr0.1_it100_max31_min10_r0.00,0.747982,0.255537,0.598858
76,gbm_lr0.1_it200_max31_min20_r0.00,0.728950,0.246795,0.594175
104,gbm_lr0.1_it200_max63_min20_r0.00,0.725490,0.253918,0.592889
55,gbm_lr0.1_it100_max63_min20_r0.02,0.737024,0.242525,0.592759
83,gbm_lr0.1_it200_max31_min20_r0.02,0.732987,0.263911,0.590530
69,gbm_lr0.1_it200_max31_min10_r0.02,0.738178,0.295031,0.589123


In [17]:
# hyperparameter tuning when biomarker features are included raw
# no longer captured in cell order
metrics = pd.DataFrame(metrics_list)
metrics.loc[metrics.institution_name == "All", ["experiment_name", "acc", "f1_pos", "roc_auc"]].sort_values(
    by="roc_auc", ascending=False
)

,experiment_name,acc,f1_pos,roc_auc
90,gbm_lr0.2_it200_max31_min10,0.906574,0.758209,0.943299
62,gbm_lr0.2_it100_max31_min10,0.907728,0.761905,0.942696
34,gbm_lr0.1_it200_max31_min10,0.907728,0.759036,0.941163
104,gbm_lr0.2_it200_max63_min10,0.906574,0.756757,0.940678
6,gbm_lr0.1_it100_max31_min10,0.908881,0.764881,0.940313
48,gbm_lr0.1_it200_max63_min10,0.905998,0.757079,0.939671
76,gbm_lr0.2_it100_max63_min10,0.908304,0.761619,0.939545
41,gbm_lr0.1_it200_max31_min20,0.904844,0.751131,0.938255
20,gbm_lr0.1_it100_max63_min10,0.908881,0.767647,0.937464
111,gbm_lr0.2_it200_max63_min20,0.904268,0.747720,0.937444


In [7]:
configs = []
for biomarker_svd_n_components in [None, 10, 50, 100, 1000]:
    config = bcs.modeling.ModelingConfig(
        experiment_name=f"gbm_svd{biomarker_svd_n_components}",
        gbm_learning_rate=0.2,
        gbm_max_iter=200,
        gbm_max_leaf_nodes=31,
        gbm_min_samples_leaf=10,
        gbm_l2_regularization=0.01,
        biomarker_feature_action="svd",
        biomarker_svd_n_components=biomarker_svd_n_components,
    )
    configs.append(config)
assert len(configs) == len({c.experiment_name for c in configs}), "Experiment names not unique."
len(configs)

6

In [8]:
model_list = []
for config in tqdm(configs, desc="SVD experiments"):
    model_evaluator = bcs.modeling.ModelEvaluator(config, mdf)
    model = model_evaluator.train_and_evaluate()
    model_list.append(model)
len(model_list)

SVD experiments:  83%|██████████████████████████████████████████████████████████████████████████████▎               | 5/6 [12:22<02:28, 148.47s/it]


ValueError: n_components=10000 must be between 0 and min(n_samples, n_features)=1038 with svd_solver='full'

In [9]:
# hyperparameter tuning with SVD
# no longer captured in cell order
metrics_list = []
for model in model_list:
    metrics_list.extend(model.metrics_)
metrics = pd.DataFrame(metrics_list)
metrics.loc[metrics.institution_name == "All", ["experiment_name", "acc", "f1_pos", "roc_auc"]].sort_values(
    by="roc_auc", ascending=False
)

,experiment_name,acc,f1_pos,roc_auc
27,gbm_svd100,0.764706,0.209302,0.613485
20,gbm_svd50,0.777970,0.067797,0.610542
13,gbm_svd10,0.764706,0.227273,0.597756
34,gbm_svd1000,0.779700,0.010363,0.593636
6,gbm_svdNone,0.777970,0.005168,0.592006
